In [1]:
from geopy.distance import distance
import geopandas as gpd
import datetime
import numpy as np

file = '/home/conor/Projects/D4D/connor_2020-02-10T13_39_00-07_00.geojson'

In [3]:
df = gpd.read_file(file)

def f_lat(x):
    b = x.bounds
    return b[1]

def f_lon(x):
    b = x.bounds
    return b[0]

def f_coords(x):
    b = x.bounds
    return (b[1], b[0])

df['lattitude'] = df['geometry'].apply(f_lat)
df['longitude'] = df['geometry'].apply(f_lon)
df['coordinates'] = df['geometry'].apply(f_coords)

df["timestamp_dt"] = df["timestamp"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))



In [5]:
df.sort_values(by="timestamp_dt", ascending=True)

,algorithm,background_maximum_return,background_mean_return,background_minimum_return,centroid_lat,centroid_lon,change_area,compactness,detect_strength,detect_type,...,target_minimum_return,target_total_return,timestamp,weighted_centroid_lat,weighted_centroid_lon,geometry,lattitude,longitude,coordinates,timestamp_dt
0,ACD-29August2019,7.82,3.15,0.02,33.066993,41.831021,329.63,1.0846,-26.4,Departure,...,101.19,6341.48,2018-11-06T00:00:00,33.067041,41.831040,POINT Z (41.83098 33.06712 0.00000),33.067117,41.830984,"(33.067117, 41.8309845)",2018-11-06
1,ACD-29August2019,23.01,8.48,1.59,33.065138,41.833288,423.81,1.0200,-25.4,Departure,...,49.94,8828.28,2018-11-06T00:00:00,33.065097,41.833273,POINT Z (41.83325 33.06526 0.00000),33.065261,41.833251,"(33.0652614, 41.8332511)",2018-11-06
2,ACD-29August2019,1.72,1.24,0.54,32.763584,42.054833,313.79,0.9193,-32.7,Departure,...,201.96,7629.75,2018-11-12T00:00:00,32.763603,42.054840,POINT Z (42.05479 32.76371 0.00000),32.763708,42.054792,"(32.763708, 42.0547923)",2018-11-12
3,ACD-29August2019,3.98,1.49,0.26,32.412282,42.699591,366.01,0.8837,-24.3,Departure,...,116.86,2231.87,2018-11-12T00:00:00,32.412269,42.699586,POINT Z (42.69955 32.41241 0.00000),32.412406,42.699551,"(32.4124063, 42.6995507)",2018-11-12
4,ACD-29August2019,4.15,2.43,0.44,32.413997,42.505508,485.80,1.4297,-27.5,Departure,...,136.28,1797.36,2018-11-17T00:00:00,32.414018,42.505498,POINT Z (42.50546 32.41412 0.00000),32.414121,42.505461,"(32.4141211, 42.5054609)",2018-11-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,ACD-29August2019,4.61,2.43,1.00,32.916986,42.109402,366.12,0.8837,33.1,Arrival,...,52.64,15781.51,2019-10-26T00:00:00,32.917054,42.109412,POINT Z (42.10936 32.91711 0.00000),32.917110,42.109361,"(32.9171097, 42.1093609)",2019-10-26
447,ACD-29August2019,7.54,2.62,0.43,32.805409,42.342746,366.06,1.0314,21.3,Arrival,...,118.11,2077.18,2019-10-26T00:00:00,32.805415,42.342735,POINT Z (42.34271 32.80553 0.00000),32.805533,42.342705,"(32.8055327, 42.3427052)",2019-10-26
449,ACD-29August2019,5.73,3.76,0.92,32.711101,42.416815,282.21,1.3880,21.5,Arrival,...,62.48,1809.24,2019-10-30T00:00:00,32.711102,42.416815,POINT Z (42.41678 32.71122 0.00000),32.711225,42.416779,"(32.7112249, 42.4167789)",2019-10-30
448,ACD-29August2019,20.65,8.79,1.01,32.312227,42.443671,376.22,0.8539,18.6,Arrival,...,69.54,2073.94,2019-10-30T00:00:00,32.312216,42.443669,POINT Z (42.44363 32.31235 0.00000),32.312350,42.443634,"(32.3123504, 42.4436342)",2019-10-30


## dts in the following cell will be a list of unique datetime objects from sample geojson file##

In [20]:
list_dts = df["timestamp_dt"].to_list()
np_dts = np.array(list_dts)
dts = np.unique(np_dts)

In [8]:
class Collect:
    def __init__(self, coords, time, num):
        self.coords = coords
        self.time = time
        self.num = num
    def print_collect(self):
        print("Date of collect: ", end = " ") 
        print(self.time)
        print("Number of points in collect: ", end = " ") 
        print(self.num)
        print("Coordinates of points: ", end = " ") 
        print(self.coords)
    def get_coords(self):
        return self.coords
    def get_time(self):
        return self.time

In [54]:
class Cluster:
    def __init__(self, coords, time, size):
        self.coords = coords
        self.time = time
        self.size = size
    def new_point(self, coord):
        self.coords.append(coord)
        self.size += 1
    def print_cluster(self):
        print("Date of cluster: ", end = " ")
        print(self.time)
        print("Size of cluster: ", end = " ")
        print(self.size)
        print("Point within collect: ")
        for coord in self.coords:
            print(coord.get_coords())
    def get_coords(self):
        return self.coords

In [10]:
# built to keep track of whether a set of coordinates is already in a cluster

class Cluster_Coord:
    def __init__(self, coords, c):
        self.coords = coords
        self.c = c
    def change_status(self, c):
        self.c = c
    def get_status(self):
        return self.c
    def get_coords(self):
        return self.coords

## The following 2 cells can be used to deconstruct the object "master" lists ##

In [62]:
collects = [] # list of collects

In [63]:
clusters = [] # list of clusters

In [64]:
def init_collect(time):
    coords = []
    collect_data = df.loc[df['timestamp_dt']==time, ['coordinates', 'timestamp_dt']]
    num = len(collect_data)
    for i in range(num):
        d = collect_data.iloc[i,:]
        coords.append(d[0])
    collects.append(Collect(coords, time, num))

In [65]:
# initialize all of the collects to the collects array - can be different time period down the line

for time in dts:
    init_collect(time)

idx = 0
for collect in collects:
    print("Index of collects array: " + str(idx))
    idx += 1
    collect.print_collect()
    print()

Index of collects array: 0
Date of collect:  2018-11-06 00:00:00
Number of points in collect:  2
Coordinates of points:  [(33.067117, 41.8309845), (33.0652614, 41.8332511)]

Index of collects array: 1
Date of collect:  2018-11-12 00:00:00
Number of points in collect:  2
Coordinates of points:  [(32.763708, 42.0547923), (32.4124063, 42.6995507)]

Index of collects array: 2
Date of collect:  2018-11-17 00:00:00
Number of points in collect:  3
Coordinates of points:  [(32.4141211, 42.5054609), (32.4138167, 42.5060704), (32.4272773, 42.5031058)]

Index of collects array: 3
Date of collect:  2018-11-22 00:00:00
Number of points in collect:  1
Coordinates of points:  [(32.3809129, 42.3340765)]

Index of collects array: 4
Date of collect:  2018-11-29 00:00:00
Number of points in collect:  5
Coordinates of points:  [(32.4711723, 42.9994236), (32.4141056, 42.505332), (32.4138138, 42.5059431), (32.4270296, 42.5024657), (32.4884621, 43.0035486)]

Index of collects array: 5
Date of collect:  2018-

## The following two cells are to show how you can find distances between the points in a collect. This is what we are looking at to form clusters; this is the threshold parameter 

In [66]:
def print_collect_distances(collect):
    coords = collect.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i]))
                print("Second point: " + str(coords[j]))
                dist = distance(coords[i], coords[j]).km
                print("Distance in km between the points: " + str(dist))
                print()

In [67]:
print_collect_distances(collects[18])

Points 0 and 1
First point: (32.1450048, 42.5223647)
Second point: (32.4538848, 42.6020192)
Distance in km between the points: 35.06431435202495

Points 0 and 2
First point: (32.1450048, 42.5223647)
Second point: (32.4538062, 42.6020405)
Distance in km between the points: 35.056229990889236

Points 0 and 3
First point: (32.1450048, 42.5223647)
Second point: (32.9990875, 42.4775261)
Distance in km between the points: 94.80863897089579

Points 0 and 4
First point: (32.1450048, 42.5223647)
Second point: (32.1500636, 42.4833821)
Distance in km between the points: 3.720214779894551

Points 0 and 5
First point: (32.1450048, 42.5223647)
Second point: (32.4538974, 42.6020054)
Distance in km between the points: 35.06540110538875

Points 0 and 6
First point: (32.1450048, 42.5223647)
Second point: (32.6542806, 42.2414206)
Distance in km between the points: 62.35463523605604

Points 0 and 7
First point: (32.1450048, 42.5223647)
Second point: (32.9991032, 42.4775166)
Distance in km between the poin

In [47]:
def find_cluster(cc): #pass in 2 cluster_coords
    for cluster in clusters:
        for point in cluster.get_coords():
            if point == cc:
                return cluster
    return 0

In [68]:
def form_clusters_by_collect(collect, km_dist):
    cc = []
    coords = collect.get_coords()
    #init Cluster_Coords
    for coord in coords:
        cc.append(Cluster_Coord(coord, False))
    for i in range(len(cc)):
        for j in range(len(cc)):
            if (j > i):
                c1 = cc[i].get_coords()
                c2 = cc[j].get_coords()
                dist = distance(c1, c2).km
                if dist <= km_dist:
                    if cc[i].get_status() == True and cc[j].get_status() == True:
                        # check if in same cluster
                        clust1 = find_cluster(cc[i])
                        clust2 = find_cluster(cc[j])
                        if clust1 != clust2:
                            # concatenate clusters
                            idx = -1
                            idx = clusters.index(clust1)
                            if idx != -1:
                                clusters[idx].extend(clust2)
                                clusters.remove(clust2)
                        # else do nothing - already in a cluster together
                        
                    elif cc[i].get_status() == True and cc[j].get_status() == False:
                        clust = find_cluster(cc[i])
                        idx = -1
                        idx = clusters.index(clust)
                        clusters[idx].new_point(cc[j])
                        cc[j].change_status(True)
                        
                    elif cc[i].get_status() == False and cc[j].get_status() == True:
                        clust = find_cluster(cc[j])
                        idx = -1
                        idx = clusters.index(clust)
                        clusters[idx].new_point(cc[i])
                        cc[i].change_status(True)
                        
                    else: 
                        # create new cluster (size of 2 because every cluster starts with 2 points)
                        clusters.append(Cluster([cc[i], cc[j]], collect.get_time(), 2))
                        cc[i].change_status(True)
                        cc[j].change_status(True)


In [93]:
clusters = []

form_clusters_by_collect(collects[18], 40)

idx = 0
for cluster in clusters:
    print("Index in clusters array: " + str(idx))
    idx += 1
    cluster.print_cluster()
    print()

Index in clusters array: 0
Date of cluster:  2019-01-21 00:00:00
Size of cluster:  5
Point within collect: 
(32.1450048, 42.5223647)
(32.4538848, 42.6020192)
(32.4538062, 42.6020405)
(32.1500636, 42.4833821)
(32.4538974, 42.6020054)

Index in clusters array: 1
Date of cluster:  2019-01-21 00:00:00
Size of cluster:  4
Point within collect: 
(32.9990875, 42.4775261)
(32.9991032, 42.4775166)
(32.9873952, 42.3647473)
(32.6542806, 42.2414206)



## As with collects, the next two cells are some code that can actually tell us distances between the points in a specific cluster. Hopefully, this can help us make informed decisions about the effectiveness of a chosen threshold. Remember that distances are in km.

In [90]:
def print_cluster_distances(cluster):
    coords = cluster.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i].get_coords()))
                print("Second point: " + str(coords[j].get_coords()))
                dist = distance(coords[i].get_coords(), coords[j].get_coords()).km
                print("Distance in km between the points: " + str(dist))
                print()

In [95]:
idx = 0
for cluster in clusters:
    print("INDEX OF CLUSTER IN ARRAY: " + str(idx))
    print_cluster_distances(cluster)
    print()
    print()

INDEX OF CLUSTER IN ARRAY: 0
Points 0 and 1
First point: (32.1450048, 42.5223647)
Second point: (32.4538848, 42.6020192)
Distance in km between the points: 35.06431435202495

Points 0 and 2
First point: (32.1450048, 42.5223647)
Second point: (32.4538062, 42.6020405)
Distance in km between the points: 35.056229990889236

Points 0 and 3
First point: (32.1450048, 42.5223647)
Second point: (32.1500636, 42.4833821)
Distance in km between the points: 3.720214779894551

Points 0 and 4
First point: (32.1450048, 42.5223647)
Second point: (32.4538974, 42.6020054)
Distance in km between the points: 35.06540110538875

Points 1 and 2
First point: (32.4538848, 42.6020192)
Second point: (32.4538062, 42.6020405)
Distance in km between the points: 0.008943451304389186

Points 1 and 3
First point: (32.4538848, 42.6020192)
Second point: (32.1500636, 42.4833821)
Distance in km between the points: 35.49584467468661

Points 1 and 4
First point: (32.4538848, 42.6020192)
Second point: (32.4538974, 42.6020054)

## Now let's have some more fun :-)
## I am now going to start making new, relevant cluster attributes in order to make visual representations of clusters.